In [4]:
import pandas as pd

df = pd.read_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv")

#remove all rows where the date column is empty
df = df[df["date"].notna()]

#date format
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df.dropna(subset="date", inplace=True)
df['date'] = df['date'].dt.date
df['date'] = pd.to_datetime(df['date'])

def replace_earliest_date(df):
    # Convert 'date' to datetime format and filter the data for January 2010
    df['date'] = pd.to_datetime(df['date'])
    jan_2010_filter = df['date'].dt.strftime('%Y-%m') == '2010-01'
    jan_2010_data = df[jan_2010_filter]

    # Find the earliest date for each stock
    earliest_dates = jan_2010_data.groupby('stock_RIC')['date'].min()

    # Replace the earliest dates with 2010-01-01
    df.loc[df.set_index(['stock_RIC', 'date']).index.isin(earliest_dates.reset_index().set_index(['stock_RIC', 'date']).index), 'date'] = pd.Timestamp('2009-12-30')

    return df

df = replace_earliest_date(df)

def get_last_observation_next_month(output_file):

    # Ensure the date column is in datetime format
    df['date'] = pd.to_datetime(df['date'])

    # Sort the DataFrame by stock and date to ensure the last observation can be found
    df.sort_values(by=['stock_RIC', 'date'], ascending=[True, False], inplace=True)

    # Group by stock and year-month, then take the last observation
    df['year_month'] = df['date'].dt.to_period('M')
    last_observation_df = df.groupby(['stock_RIC', 'year_month']).agg('last').reset_index()

    # Change the date to the first day of the next month
    last_observation_df['date'] = (last_observation_df['date'] + pd.offsets.MonthEnd(1)) + pd.Timedelta(days=1)

    # Drop the year_month column as it's no longer needed
    last_observation_df.drop('year_month', axis=1, inplace=True)    

    # Save the resulting DataFrame to a new CSV file
    last_observation_df.to_csv(output_file, index=False)
    print(f"Processed data has been saved to {output_file}")

# Specify the path to your CSV file and the path for the output file
#input_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv'
output_file = '/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/m_stock_level_data.csv'

# Call the function to process the file 
get_last_observation_next_month(output_file)


display(df)

/var/folders/m9/n85vv59j4lj5g5nndc07m8t40000gn/T/ipykernel_4572/220369037.py:3: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv")


Processed data has been saved to /Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/m_stock_level_data.csv


,Unnamed: 0,stock_RIC,date,price,return1D,return1Wk,return1Mo,volume,turnover,market_cap,gross_profit,price_to_BV,bid_price,ask_price,year_month
1185671,1414640,0MW4EUR.xbo^K15,2015-11-03,14.970000,-0.133422,0.133779,-6.675567e-02,938065.0,1.404420e+07,7.302660e+09,2.235776e+09,NaN,14.760000,15.190000,2015-11
1185672,1414641,0MW4EUR.xbo^K15,2015-11-03,14.970000,-0.133422,0.066845,-6.675567e-02,0.0,1.404420e+07,7.305600e+09,2.235776e+09,NaN,14.760000,15.190000,2015-11
1185673,1414642,0MW4EUR.xbo^K15,2015-11-03,14.970000,-0.133422,0.066845,-5.705210e-10,0.0,1.404420e+07,7.298249e+09,2.235776e+09,NaN,14.760000,15.190000,2015-11
1185674,1414643,0MW4EUR.xbo^K15,2015-11-03,14.970000,-0.133422,0.133779,6.684492e-02,0.0,1.404420e+07,7.264478e+09,2.235776e+09,NaN,14.760000,15.190000,2015-11
1185675,1414644,0MW4EUR.xbo^K15,2015-11-03,14.970000,-0.133422,-0.133422,1.337793e-01,0.0,1.404420e+07,7.266316e+09,2.235776e+09,NaN,14.760000,15.190000,2015-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15252,18237,ZURN.S,2010-01-08,157.583688,0.649632,2.604857,5.110810e+00,770634.0,1.213405e+08,2.323664e+10,NaN,1.148270,157.448074,157.583688,2010-01
15251,18236,ZURN.S,2010-01-07,156.053008,-1.366937,1.942605,3.962179e+00,835262.0,1.305151e+08,2.301093e+10,NaN,1.129708,155.985423,156.053008,2010-01
15250,18235,ZURN.S,2010-01-06,158.087559,-0.382979,3.355408,4.369148e+00,659170.0,1.040779e+08,2.331094e+10,NaN,1.151941,158.087559,158.155089,2010-01
15249,18234,ZURN.S,2010-01-05,158.256072,2.173913,3.115401,4.770397e+00,974130.0,1.533522e+08,2.333579e+10,NaN,1.149657,158.256072,158.323415,2010-01


In [1]:
import pandas as pd

df = pd.read_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv")
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['date'] = df['date'].dt.date
df.dropna(subset=['date'], inplace=True)
df['date'] = pd.to_datetime(df['date'], errors='coerce')

#### drop all dates in 2014-09
df = df[(df['date'].dt.year != 2014) | (df['date'].dt.month != 9)] #2014-10-01 anomalie
df = df[(df['date'].dt.year != 2019) | (df['date'].dt.month != 6)] #2019-07-01 anomalie 


####### append new data by running script

print(df.dtypes)
df.to_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv")

stock_RIC               object
date            datetime64[ns]
price                  float64
return1D               float64
return1Wk              float64
return1Mo              float64
volume                 float64
turnover               float64
market_cap             float64
gross_profit           float64
price_to_BV            float64
bid_price              float64
ask_price              float64
dtype: object
